<a href="https://colab.research.google.com/github/Argentan/DMA_LAB2/blob/master/tutoriales/08_Ejemplo_CNN_MNIST.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Digit Recognizer
### Learn computer vision fundamentals with the famous MNIST data

Los datos son obtenidos de https://www.kaggle.com/c/digit-recognizer

In [1]:
##
import pandas as pd 
import numpy as np 
import seaborn as sns

from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras.callbacks import EarlyStopping

import matplotlib.pyplot as plt
%matplotlib notebook

Leemos los datos

In [2]:
X_train = pd.read_csv("https://github.com/Argentan/DMA_LAB2/blob/master/data/Digit_Recognizer.csv.zip?raw=true", compression='zip')
y_train = X_train.label
X_train = X_train.drop("label", axis=1)

In [3]:
X_train.head()

,pixel0,pixel1,pixel2,pixel3,pixel4,pixel5,pixel6,pixel7,pixel8,pixel9,...,pixel774,pixel775,pixel776,pixel777,pixel778,pixel779,pixel780,pixel781,pixel782,pixel783
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [4]:
28*28

784

In [5]:
X_train.shape

(42000, 784)

Distribución del Target

In [ ]:
sns.countplot(y_train)

El valor máximo que puede tomar un pixel es

In [ ]:
X_train.max().max()

Normalizamos los datos entre 0 y 1

In [ ]:
X_train /= X_train.max().max()
X_train.head()

Hacemos un `reshape` al tamaño correct de las imagenes

In [ ]:
X_train.values.reshape(-1,28,28,1).shape

In [ ]:
X_train = X_train.values.reshape(-1,28,28,1)

Imprimimos una imagen de ejemplo

In [ ]:
plt.imshow(X_train[3][:,:,0])

In [ ]:
plt.imshow(X_train[13][:,:,0], cmap='binary')

In [ ]:
fig, ax = plt.subplots(8, 8, figsize=(6, 6))
for i, axi in enumerate(ax.flat):
    axi.imshow(X_train[i, :, :, 0], cmap='binary')
    axi.set(xticks=[], yticks=[])

Separamos en Train y Test

In [ ]:
test_idx = np.random.choice(range(X_train.shape[0]), int(X_train.shape[0] * 0.1), replace=False)
train_idx = [i for i in range(X_train.shape[0]) if i not in test_idx]
X_test = X_train[test_idx]
y_test = pd.get_dummies(y_train[test_idx])
X_train = X_train[train_idx]
y_train = pd.get_dummies(y_train[train_idx])

In [ ]:
y_train.tail()

## Deep Neural Networks - Convolutional Neural Networks 

Un red neuronal "profunda" es aquella que tiene múltiples representaciones latentes que buscan extraer variables explicativas de bajo nivel y componerlas en capas superiores. Las redes convolucionales son un tipo de estas redes profundas, puntualmente utiles para la clasificacion de imagenes. 

###  Convolutional Neural Networks 


![](http://www.mdpi.com/information/information-07-00061/article_deploy/html/images/information-07-00061-g001.png)


**Convolutional layer: ** Esta capa utiliza un filtro convolutivo para procesar partes de la imagen, el filtro consiste de coeficiente que se aplican sucesivamente a pedasos de la imagen.

![](https://gblobscdn.gitbook.com/assets%2F-LIA3amopGH9NC6Rf0mA%2F-M4bJ-IWAKzglR0XHFwU%2F-M4bJ4XuAd99mcti2Q2K%2Fconv_no_padding.gif?alt=media) 

**Tranco (stride): ** El tranco es la cantidad de pixeles que se saltean cada vez qeu se mueve el filtro.

**Padding: ** Margen que dejamos para ajustar el recuadro de la convolución

![](https://gblobscdn.gitbook.com/assets%2F-LIA3amopGH9NC6Rf0mA%2F-M4bJ-IWAKzglR0XHFwU%2F-M4bJ4XwDlK64zz3I0ZH%2Fconv_padding.gif?alt=media)

Que aprende la convolución?

![](https://ujwlkarn.files.wordpress.com/2016/08/giphy.gif) 


**Pooling Layer:**  Esta capa se utiliza para filtrar variables menos importantes y disminuir la cantidad necesaria de parámetros totales. 

![](https://upload.wikimedia.org/wikipedia/commons/e/e9/Max_pooling.png)



Definimos el Modelo

In [ ]:
model = Sequential()

model.add(Conv2D(32, kernel_size=(3, 3), activation='relu', input_shape = X_train[0].shape))
model.add(Conv2D(32, kernel_size=(3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.25))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(Conv2D(64, (3, 3), activation='relu'))
model.add(MaxPooling2D(pool_size=(2, 2), strides=(2,2)))
model.add(Dropout(0.25))
model.add(Conv2D(128, (4, 4), activation='relu'))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(128, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.25))
model.add(Dense(y_train.shape[1], activation='softmax'))
model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
model.summary()

In [ ]:
(3 * 3 * 3 + 1) * 32

Entrenamos el Modelo

In [ ]:
model.fit(X_train, y_train, batch_size= 320 , epochs= 30, verbose= 1, validation_split=0.1,
          callbacks=[EarlyStopping(monitor='val_acc', patience=3,
                                   verbose=1, mode='auto', restore_best_weights=True)])

Epoch 1/30
107/107 [==============================] - 77s 722ms/step - loss: 0.0678 - accuracy: 0.9828 - val_loss: 0.0522 - val_accuracy: 0.9847
Epoch 2/30
107/107 [==============================] - 77s 720ms/step - loss: 0.0562 - accuracy: 0.9857 - val_loss: 0.0433 - val_accuracy: 0.9884
Epoch 3/30
107/107 [==============================] - 77s 722ms/step - loss: 0.0475 - accuracy: 0.9877 - val_loss: 0.0434 - val_accuracy: 0.9886
Epoch 4/30
107/107 [==============================] - 77s 724ms/step - loss: 0.0430 - accuracy: 0.9886 - val_loss: 0.0295 - val_accuracy: 0.9921
Epoch 5/30
107/107 [==============================] - 77s 724ms/step - loss: 0.0401 - accuracy: 0.9892 - val_loss: 0.0300 - val_accuracy: 0.9939
Epoch 6/30
107/107 [==============================] - 78s 725ms/step - loss: 0.0396 - accuracy: 0.9908 - val_loss: 0.0323 - val_accuracy: 0.9923
Epoch 7/30
107/107 [==============================] - 78s 724ms/step - loss: 0.0353 - accuracy: 0.9914 - val_loss: 0.0341 - val_ac

In [ ]:
model.predict(X_test)

array([[1.9245449e-04, 9.1602085e-03, 2.9198302e-02, ..., 9.4797713e-01,
        1.2794309e-03, 2.0388423e-03],
       [1.0000000e+00, 4.8336786e-14, 1.0786390e-09, ..., 1.7724833e-12,
        1.8289879e-09, 2.0860993e-08],
       [9.9998927e-01, 2.5031799e-10, 2.0103080e-07, ..., 4.0457189e-09,
        5.5021462e-07, 8.4001749e-06],
       ...,
       [2.6923713e-13, 1.0000000e+00, 5.6628102e-09, ..., 7.2258723e-09,
        4.0454382e-09, 7.9959699e-11],
       [8.6122600e-15, 1.0000000e+00, 5.1273497e-10, ..., 7.5875517e-10,
        3.8951281e-10, 5.4037304e-12],
       [2.0352740e-11, 1.0098380e-12, 4.9106034e-14, ..., 3.9597150e-13,
        2.2960729e-10, 4.2456274e-09]], dtype=float32)

Evaluamos la Predicción

In [ ]:
preds = model.predict(X_test).argmax(axis=1)
pd.crosstab(y_test.idxmax(axis=1), preds)

col_0,0,1,2,3,4,5,6,7,8,9
row_0,,,,,,,,,,
0,397,0,0,0,0,0,1,0,2,1
1,0,451,0,0,0,0,1,0,0,0
2,0,0,392,0,0,1,0,3,1,0
3,0,0,3,439,0,3,0,1,1,0
4,0,2,0,0,447,0,0,2,0,2
5,0,0,0,0,0,372,2,0,0,0
6,0,0,0,0,0,1,414,0,1,0
7,0,1,4,0,0,0,0,428,0,0
8,0,1,0,0,0,3,0,0,395,1


## Aumentar el Dataset con transformaciones de Imagenes

In [ ]:
X_train.shape

In [ ]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=10,  # Rotar aleatoriamente la imagen en un 10%
        zoom_range = 0.1, # Hace zoom aleatoriamente la imagen en un 10%
        width_shift_range=0.1,  # Hacer las imagenes aleatoriamente un 10% más anchas
        height_shift_range=0.1,  # Hacer las imagenes aleatoriamente un 10% más altas
        horizontal_flip=False,  # Rotar Horizontalmente
        vertical_flip=False)  # Rotar Verticalmente


datagen.fit(X_train)

In [ ]:
X_train.shape

In [ ]:
model.fit_generator(datagen.flow(X_train,y_train, batch_size=320),
                              epochs = 30, verbose = 1, steps_per_epoch= X_train.shape[0] // 320)

In [ ]:
preds = model.predict(X_test).argmax(axis=1)
pd.crosstab(y_test.idxmax(axis=1), preds)

## Aplicar un Modelo Pre-entrenado 

Vamos a utilizar `resnet50` como una capa de nuestra red

In [ ]:
from tensorflow.keras.applications.resnet50 import ResNet50

new_model = Sequential()
new_model.add(ResNet50(include_top=False, weights='imagenet', pooling='avg'))
new_model.add(Dense(y_train.shape[1], activation='softmax'))
new_model.compile(optimizer='rmsprop',loss='categorical_crossentropy', metrics=['accuracy'])
new_model.summary()

In [ ]:
new_model.fit(np.repeat(X_train, 3, -1), y_train, batch_size=320, epochs=30, verbose=1, validation_split=0.1,
              callbacks=[EarlyStopping(monitor='val_acc', patience=3, verbose=1, mode='auto', restore_best_weights=True)])